## Workbook of tonal harmony
#### Analysis of the string quartet Op. 59 n.3 mov. 1 (Introduzione} from music21

In [1]:
from pcsPy import *
import pickle, copy
from IPython.display import Image 
sys.path.append('../')
from tonalHarmonyDefs import enharmonicDictionary, shortHands, scoreAnalysis, tonalPartition, showAnalysis

<IPython.core.display.Javascript object>

Here we import the tonal model we are going to use for the analysis. A tonal model is the matrix of all the operators that connect the chords of a chord library as roman numerals.
Typical models are built using combinations of basic chord symbols and possible extension. 

Use tonalHarmonyModel.py to generate or modify models

In [2]:
score = 'Op59n3m1_Intro.musicxml'
seq,chords = readScore(score,music21=False,show=False)

In [ ]:
# To write chords and corresponding operators - uses musescore
WRITEscoreOps(seq,w=True)

In [3]:
dnodes,dedges,dcounts,deg,modul,Gx,Gxu = scoreNetwork(seq,ntx=True,general=True)
print('average degree = ',deg,' modularity = ',modul)
# Write to csv for visualization in Gephi or other network drawing software (see figure below)
# dnodes.to_csv('nodesOp59n3m1.csv')
# dedges.to_csv('edgesOp59n3m1.csv')
# Get modularity scores
# part = cm.best_partition(Gxu) #,resolution=0.4)

average degree =  1.2608695652173914  modularity =  0.5608


## ![title](Op59n3m1.png)

In [4]:
moduldict,modul = tonalPartition(dnodes,Gx,Gxu)
# modul.to_csv('~/Desktop/modul.csv')

In [ ]:
modul[['Label','Degree','Modularity']].query('Modularity == 2').sort_values(by=['Degree'],ascending=False)

In [5]:
# Dictionary of modularity classes and relative keys
keydict = {0:'B-', 1:'G', 2:'G', 3:'C', 4:'C', 5:'A'}

In [6]:
# clean non harmonic grace note at [20]
seq[20] = [2, 5, 8, 11]
seq[21] = [2, 5, 8, 11]
print('new sequence:',seq[19],seq[20],seq[21],seq[22])

new sequence: [3, 7, 10] [2, 5, 8, 11] [2, 5, 8, 11] [2, 5, 8, 11]


In [7]:
# manual control of modulations
keychange = {17:'E-'}
# manual control of roman numerals
altrn = None #{2:'II'}

In [8]:
nxt,rn,ops = scoreAnalysis(seq,moduldict,keydict,keychange=keychange,altrn=altrn,table='../tonal.harmony.minimal')

modulation at or before chord no.  1
modulation at or before chord no.  9
modulation at or before chord no.  14
modulation at or before chord no.  17
modulation at or before chord no.  30
modulation at or before chord no.  34


In [9]:
analysis = showAnalysis(nxt,chords,seq,rn,ops,keydict,moduldict,last=True,display=False)

In [10]:
analysis

,pcs,chord,rn,ops,key,modul
0,"[0, 3, 6, 9]",CE-F#A,viio7,"O(0,0,0,0)",B-,0.0
1,"[0, 3, 6, 9]",CE-F#A,iio7,"O(0,0,-1,0)",B-,0.0
2,"[9, 0, 3, 5]",ACE-F,V7,"O(0,0,0,0)",B-,0.0
3,"[9, 0, 3, 5]",ACE-F,V7,"O(0,0,1,-1)",B-,0.0
4,"[9, 0, 4]",ACE,vii,"O(2,2,0,0)",B-,0.0
5,"[6, 9, 0, 2]",F#ACD,III7,"O(0,0,0,0)",G,1.0
6,"[6, 9, 0, 2]",F#ACD,III7,"O(0,0,-1,0)",G,1.0
7,"[6, 9, 11, 2]",F#ABD,IIIb7,"O(0,0,1,0)",G,1.0
8,"[6, 9, 0, 2]",F#ACD,V7,"O(0,2,-1,0)",G,1.0
9,"[11, 2, 6]",BDF#,iii,"O(0,0,1)",G,1.0
